# Data Cleaning code for Marriage and Divorce Rate Charts

In [51]:
import pandas as pd
import json
from datetime import datetime

In [93]:
# Importing datasets
marriage = pd.read_csv('../data/state-marriage-rates.csv')
divorce = pd.read_csv('../data/state-divorce-rates.csv')
sah = pd.read_csv('../data/stay-at-home.csv')
fips = pd.read_csv('../data/us-state-ansi-fips.csv')

In [88]:
# Convert date objects
sah['Date enacted'] = pd.to_datetime(sah['Date enacted']).dt.date
sah['Date lifted'] = pd.to_datetime(sah['Date lifted']).dt.date

def unix_stamp(date):
    return  datetime.strptime(date,"%Y-%m-%d").date()

sah['Group'] = ""

for i, val in enumerate(sah['State']):
    if sah['Date enacted'][i] < unix_stamp("2020-03-22"):
        sah['Group'][i] = 'Came into effect before March 22'
    elif unix_stamp("2020-03-22") <= sah['Date enacted'][i] < unix_stamp("2020-03-29"):
        sah['Group'][i] = 'Came into effect before March 29'
    elif unix_stamp("2020-03-29") <= sah['Date enacted'][i] < unix_stamp("2020-04-05"):
        sah['Group'][i] = 'Came into effect before April 5'
    elif unix_stamp("2020-04-05") <= sah['Date enacted'][i] < unix_stamp("2020-04-12"):
        sah['Group'][i] = 'Came into effect before April 12'
    else:
        sah['Group'][i] = "No Statewide order"

#Merge to get Fips codes
full_sah = sah.merge(fips, left_on='State', right_on='stname')

#Remove extraneous variables and rename
del full_sah[" stusps"]
del full_sah["stname"]
full_sah = full_sah.rename(columns={" st": "id"})

#Adding leading zeroes to 
full_sah['id'] = full_sah['id'].astype(str).apply(lambda x: x.zfill(2))


#Export the map data
full_sah.to_csv('../data/sah_map_data.csv', index=False)

C:\Users\kilaf\AppData\Local\Temp\ipykernel_22092\1323390441.py:11: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  if sah['Date enacted'][i] < unix_stamp("2020-03-22"):
C:\Users\kilaf\AppData\Local\Temp\ipykernel_22092\1323390441.py:13: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  elif unix_stamp("2020-03-22") <= sah['Date enacted'][i] < unix_stamp("2020-03-29"):
C:\Users\kilaf\AppData\Local\Temp\ipykernel_22092\1323390441.py:15: FutureWarning: Comparison of NaT with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable.
  elif unix_stamp("2020-03-29") <= sah['Date enacted'][i] < unix_stamp("2020-04-05"):
C:\Users\kilaf\AppData\Local\Temp\ipyker

In [94]:
## Data for Slope Graph
# Will need to take data from wide to long 

def slope_data_clean(df, type):
    '''
    Function to clean data for use in Observables slope chart infrastructure.
    '''

    if type == "Marriage":
        val_name = 'mar_rate'
        path_name = '../data/marriage_long.json'
    elif type == "Divorce":
        val_name = 'div_rate'
        path_name = '../data/divorce_long.json'

        #Melt the wide data to long
    df_long = pd.melt(df, id_vars='State', var_name='year', value_name=val_name)

        #Convert year to date
    df_long['year'] = pd.to_numeric(df_long['year'])
    df_long[val_name] = pd.to_numeric(df_long[val_name])

        # Filter 2019 and 2020
    df_long = df_long[df_long.year >= 2019]   
        #
    df_long = df_long.to_dict('records')

    with open(path_name, 'w') as fp:
        json.dump(df_long, fp)


In [95]:
## Running Slope clean
slope_data_clean(marriage, "Marriage")
slope_data_clean(divorce, "Divorce")



In [96]:
marriage

,State,1990,1995,1999,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Alabama,10.6,9.8,10.8,10.1,9.4,9.9,9.6,9.4,9.2,...,8.4,8.2,7.8,7.8,7.4,7.1,7.0,6.8,6.7,7.2
1,Alaska,10.2,9.0,8.6,8.9,8.1,8.3,8.1,8.5,8.2,...,7.8,7.2,7.3,7.5,7.4,7.1,6.9,6.7,6.5,5.7
2,Arizona,10.0,8.8,8.2,7.5,7.6,6.7,6.5,6.7,6.6,...,5.7,5.6,5.4,5.8,5.9,5.9,5.8,5.5,5.3,4.9
3,Arkansas,15.3,14.4,14.8,15.4,14.3,14.3,13.4,13.4,12.9,...,10.4,10.9,9.8,10.1,10.0,9.9,9.5,8.9,8.4,7.8
4,California,7.9,6.3,6.4,5.8,6.5,6.2,6.1,6.4,6.4,...,5.8,6.0,6.5,6.4,6.2,6.5,6.3,6.0,5.7,3.2
5,Colorado,9.8,9.0,8.2,8.3,8.2,8.0,7.8,7.4,7.6,...,7.0,6.8,6.5,7.1,6.8,7.4,7.3,7.6,7.3,6.7
6,Connecticut,7.9,6.6,5.8,5.7,5.4,5.7,5.5,5.8,5.8,...,5.5,5.2,5.0,5.4,5.3,5.6,5.6,5.3,5.0,4.3
7,Delaware,8.4,7.3,6.7,6.5,6.5,6.4,6.0,6.1,5.9,...,5.2,5.8,6.6,6.0,5.7,5.6,5.5,5.2,5.0,4.4
8,District of Columbia,8.2,6.1,6.6,4.9,6.2,5.1,5.1,5.2,4.1,...,8.7,8.4,10.8,11.8,8.2,8.1,8.2,7.8,7.8,6.5
9,Florida,10.9,9.9,8.7,8.9,9.3,9.4,9.0,9.0,8.9,...,7.4,7.2,7.0,7.3,8.2,8.1,7.8,7.3,7.1,5.7


In [92]:
# Merging data for grouping

group_df = pd.DataFrame({'State': marriage['State'], 'mar_rate_19': marriage['2019'], 
                        'mar_rate_20': marriage['2020'], 'div_rate_19': divorce['2019'],
                        'div_rate_20': divorce['2020'], 'group': full_sah['Group']})

group_df.to_csv('../data/scatter_data.csv', index=False)



In [50]:
## Data for US divorce and marriage rate line chart

us = pd.read_csv('../data/national-mar-div-rates.csv')

us = pd.melt(us, id_vars='Year')

us.to_csv('../data/us_mar_div.csv',index=False)